# Polymarket Basic Usage
No authentication required. Just install dr-manhattan (pip install -e .)
Full set-up in instructions dr-manhattan/README.md 

### Connect to the exchange
If imports fail after installing, restart the kernel and ensure the notebook kernel matches the environment where you installed dr_manhattan

In [36]:
from dr_manhattan.exchanges.polymarket import Polymarket
from pathlib import Path
from datetime import datetime
import pandas as pd

# Initialize public, read-only client
pm_exchange = Polymarket({"verbose": True, "timeout": 30})

try: # Sanity check: perform a lightweight public API call
    _ = pm_exchange.search_markets(limit=1, closed=False, log=False)
    print("Polymarket exchange client initialized (public mode).")
except Exception as e:
    print("Failed to initialize Polymarket exchange client.")
    raise

Polymarket exchange client initialized (public mode).


### Fetch markets and Token IDs by slug or URL

Use `fetch_markets_by_slug()` to translate a given market **slug or URL**
into its respective **YES** and **NO** token IDs.

In [2]:
# Both of these work:
slug_or_url = "https://polymarket.com/event/oscars-2026-best-actor-winner" #URL
#slug_or_url = "oscars-2026-best-actor-winner" #slug

# call the function
markets = pm_exchange.fetch_markets_by_slug(slug_or_url)

len(markets), markets[0]

# print first 3 markets as example
for i, m in enumerate(markets[:3]): 
    token_ids = m.metadata.get("clobTokenIds", [])
    readable = m.metadata.get("readable_id", None)
    print(f"[{i:02d}] id={m.id}")
    print(f"     question={m.question[:120]!r}")
    print(f"     outcomes={m.outcomes}")
    print(f"     token_ids={token_ids}")
    print(f"     readable_id={readable}")
    print()

[00] id=614007
     question='Will Leonardo DiCaprio win Best Actor at the 98th Academy Awards?'
     outcomes=['Yes', 'No']
     token_ids=['47550074058052867705919298945884495140337113150294201612719307847327469453717', '80220087096412300979031228017736830811830254178021577450197694232897885103349']
     readable_id=['oscars-2026-best-actor-winner', '614007']

[01] id=614008
     question='Will Timothée Chalamet win Best Actor at the 98th Academy Awards?'
     outcomes=['Yes', 'No']
     token_ids=['58685817664292296726620290139440291356982179626401042907081562589316381635900', '99816109404712157590087136348402913231668662708612936491417855953287910104738']
     readable_id=['oscars-2026-best-actor-winner', '614008']

[02] id=614009
     question='Will Daniel Day-Lewis win Best Actor at the 98th Academy Awards?'
     outcomes=['Yes', 'No']
     token_ids=['71477943672109190698273552742629210815721246902508944064803873551849355525684', '702149851329367686297721362270076985508023868721

# Select a Market and Inspect Its Order Book

- use `_lookup_token_id()` to find YES/NO TokenID for a given market. 
- do not assume token_id ordering -> always resolve by outcome name ("YES" / "NO")

In [3]:
# Pick a market (either by index or other criteria)
# Pick by question text:
market = next(
    m for m in markets
    if m.question == "Will Timothée Chalamet win Best Actor at the 98th Academy Awards?"
)

# Or by Index, here pick the first one:
#market = markets[0]

# Resolve token IDs for YES / NO outcomes
yes_token_id = pm_exchange._lookup_token_id(market, "Yes")
no_token_id = pm_exchange._lookup_token_id(market, "No")

print("question:", market.question)
print("YES token:", yes_token_id)
print("NO  token:", no_token_id)

question: Will Timothée Chalamet win Best Actor at the 98th Academy Awards?
YES token: 58685817664292296726620290139440291356982179626401042907081562589316381635900
NO  token: 99816109404712157590087136348402913231668662708612936491417855953287910104738


- load orderbook `get_orderbook()`

In [4]:
# fetch orderbook for YES
orderbook = pm_exchange.get_orderbook(yes_token_id)
print(orderbook.keys())  # shows available keys

dict_keys(['market', 'asset_id', 'timestamp', 'hash', 'bids', 'asks', 'min_order_size', 'tick_size', 'neg_risk', 'last_trade_price'])


- use `normalize_orderbook_levels()` to convert raw book levels into sorted [(price, size)] lists.
- compute **best bid** / **ask**, **mid price**, **depth**, and **notionals**

In [6]:
# Normalize
bids = sorted(pm_exchange.normalize_orderbook_levels(orderbook.get("bids")), key=lambda x: x[0], reverse=True) # highest bid first
asks = sorted(pm_exchange.normalize_orderbook_levels(orderbook.get("asks")), key=lambda x: x[0]) # lowest ask first            

# compute best bid, best ask, mid price
best_bid = max((px for px, _ in bids), default=None)
best_ask = min((px for px, _ in asks), default=None)
mid = (best_bid + best_ask) / 2 if (best_bid is not None and best_ask is not None) else None

# find topN depth and notionals
topN = 5
bid_depth = sum(sz for _, sz in bids[:topN])
ask_depth = sum(sz for _, sz in asks[:topN])

# find best bid/ask sizes
if best_bid is not None:
    best_bid_sz = sum(sz for px, sz in bids if px == best_bid)
else:
    best_bid_sz = None

if best_ask is not None:
    best_ask_sz = sum(sz for px, sz in asks if px == best_ask)
else:
    best_ask_sz = None

# compute notionals at level 1
bid_notional_l1 = best_bid * best_bid_sz if (best_bid is not None and best_bid_sz is not None) else 0
ask_notional_l1 = best_ask * best_ask_sz if (best_ask is not None and best_ask_sz is not None) else 0

# compute notionals at topN
bid_notional_topN = sum(px * sz for px, sz in bids[:topN])
ask_notional_topN = sum(px * sz for px, sz in asks[:topN])

# Print results
print("best_bid:", best_bid)
print("mid:", round(mid,2) if mid is not None else None)
print("best_ask:", best_ask)
print()
print("best_bid_size:", best_bid_sz)
print("best_ask_size:", best_ask_sz)
print()
print(f"l1_bid_notional(≈$):", round(bid_notional_l1,2))
print(f"l1_ask_notional(≈$):", round(ask_notional_l1,2))
print()
print(f"top{topN}_bid_depth(shares):", round(bid_depth,2))
print(f"top{topN}_ask_depth(shares):", round(ask_depth,2))
print()
print(f"top{topN}_bid_notional(≈$):", round(bid_notional_topN,2))
print(f"top{topN}_ask_notional(≈$):", round(ask_notional_topN,2))

best_bid: 0.78
mid: 0.79
best_ask: 0.79

best_bid_size: 302.69
best_ask_size: 90.04

l1_bid_notional(≈$): 236.1
l1_ask_notional(≈$): 71.13

top5_bid_depth(shares): 6330.0
top5_ask_depth(shares): 7927.68

top5_bid_notional(≈$): 4859.23
top5_ask_notional(≈$): 6476.96


### Print the Order Book:

In [8]:
# Print order book levels
MAX_LEVELS = 10 # number of levels to print

print("ORDER BOOK — YES outcome")
print(f"Market: {market.question}")
print("-" * 60)

print(f"\nTOP {MAX_LEVELS} BIDS:")
print(" Price  |    Size    |    Volume    |   CumSize   |   CumVolume ")
print("------- | ---------- | ------------ | ----------- | ------------")
cum_size = 0.0
cum_volume = 0.0
for px, sz in bids[:MAX_LEVELS]:
    print(f"{px:>6.3f}  |{sz:>10.2f}  |{px * sz:>10.2f}    |{cum_size + sz:>11.2f}  |{cum_volume + px * sz:>12.2f}")
    cum_size += sz
    cum_volume += px * sz

print(f"\nTOP {MAX_LEVELS} ASKS:")
print(" Price  |    Size    |    Volume    |   CumSize   |   CumVolume ")
print("------- | ---------- | ------------ | ----------- | ------------")
cum_size = 0.0
cum_volume = 0.0
for px, sz in asks[:MAX_LEVELS]:
    print(f"{px:>6.3f}  |{sz:>10.2f}  |{px * sz:>10.2f}    |{cum_size + sz:>11.2f}  |{cum_volume + px * sz:>12.2f}")
    cum_size += sz
    cum_volume += px * sz

ORDER BOOK — YES outcome
Market: Will Timothée Chalamet win Best Actor at the 98th Academy Awards?
------------------------------------------------------------

TOP 10 BIDS:
 Price  |    Size    |    Volume    |   CumSize   |   CumVolume 
------- | ---------- | ------------ | ----------- | ------------
 0.780  |    302.69  |    236.10    |     302.69  |      236.10
 0.770  |   5582.91  |   4298.84    |    5885.60  |     4534.94
 0.760  |    100.00  |     76.00    |    5985.60  |     4610.94
 0.740  |     16.00  |     11.84    |    6001.60  |     4622.78
 0.720  |    328.40  |    236.45    |    6330.00  |     4859.23
 0.710  |     35.15  |     24.96    |    6365.15  |     4884.18
 0.700  |   3600.65  |   2520.45    |    9965.80  |     7404.64
 0.680  |    100.00  |     68.00    |   10065.80  |     7472.64
 0.660  |   3850.00  |   2541.00    |   13915.80  |    10013.64
 0.640  |    500.00  |    320.00    |   14415.80  |    10333.64

TOP 10 ASKS:
 Price  |    Size    |    Volume    |   Cu

### Or as vertical book

In [33]:
def print_orderbook_vertical(
    bids,
    asks,
    *,
    max_levels=20,
    price_fmt="{:>6.3f}",
    size_fmt="{:>10.2f}",
):
    """
    Print a classic vertical order book:
      - asks on top (closest to mid first)
      - bids at the bottom (closest to mid first)
      - asks indented to the right

    bids: list[(price, size)] sorted DESC by price
    asks: list[(price, size)] sorted ASC by price
    """

    # Limit depth for readability
    bids = bids[:max_levels]
    asks = asks[:max_levels]

    best_bid = bids[0][0] if bids else None
    best_ask = asks[0][0] if asks else None
    mid = (best_bid + best_ask) / 2 if (best_bid and best_ask) else None

    # --- ASKS ---
    print("ASK:                 PRICE |      SIZE")
    print("-" * 45)
    for px, sz in reversed(asks):
        print(
            " " * 20
            + price_fmt.format(px)
            + " | "
            + size_fmt.format(sz)
        )

    # --- MID ---
    print("-" * 45)
    if mid is not None:
        print(f"{'MID':>20} {mid:>6.3f}")
    else:
        print(f"{'MID':>20} {'N/A':>6}")

    print("-" * 45)

    # --- BIDS ---
    for px, sz in bids:
        print(
            size_fmt.format(sz)
            + " | "
            + price_fmt.format(px)
        )

# Print classic book
print(f"\nORDER BOOK — YES outcome")
print(f"Market: {market.question}")

print_orderbook_vertical(bids, asks, max_levels=MAX_LEVELS)


ORDER BOOK — YES outcome
Market: Will Leonardo DiCaprio win Best Actor at the 98th Academy Awards?
ASK:                 PRICE |      SIZE
---------------------------------------------
                     0.920 |     250.00
                     0.910 |      60.00
                     0.900 |      10.00
                     0.860 |    1600.00
                     0.850 |    3974.47
                     0.840 |    2310.71
                     0.830 |       7.00
                     0.810 |    4307.67
                     0.800 |    1212.26
                     0.790 |      90.04
---------------------------------------------
                 MID  0.785
---------------------------------------------
    302.69 |  0.780
   5582.91 |  0.770
    100.00 |  0.760
     16.00 |  0.740
    328.40 |  0.720
     35.15 |  0.710
   3600.65 |  0.700
    100.00 |  0.680
   3850.00 |  0.660
    500.00 |  0.640


## Market Search

- use `fetch_markets()` get all active markets (up to pre-defined limit)

In [13]:
limit = 10 # how many to fetch (here limited to 10 for demo purposes)

all_active = pm_exchange.fetch_markets({"active": True, "limit": limit}) 
for m in all_active[:limit]:
    token_ids = m.metadata.get("clobTokenIds", [])
    print(m.question[:80])

✓ Fetched 10 markets from CLOB API (sampling-markets)
Cap on gambling loss deductions repealed by March 31?
Will Doug Jones win the 2026 Alabama Governor Democratic primary election?
Will the Republicans win the Vermont governor race in 2026?
Will Discord’s market cap be between $15B and $20B at market close on IPO day?
Will Xavier Becerra win the California Governor Election in 2026?
Will Tommy Tuberville win the 2026 Alabama Governor Republican primary election?
Will Ivanka Trump announce a presidential run before 2027?
Will Waymo operate in 6 cities on June 30 2026?
Will Charli XCX perform during the Super Bowl LX halftime show?
Will Chloé Zhao be nominated for Best Director at the 98th Academy Awards?


- use `search_markets()` to find markets by keyword i.e "Bitcoin"
- fetches N markets from Gamma, then filters locally. If you dont find your market, limit is probably too small (especially for older topics)

In [14]:
query="bitcoin" # User-input search term


query = query.lower()
results = pm_exchange.search_markets(
    query=query,
    limit=2000,     # fetch 5000 markets from Gamma, then filter client-side
    closed=False,   # open markets only (use None for open + closed)
    log=False,
)

number_of_matches = len(results)

print(f"\nSearch results for {query} (open markets only)")
print(f"Matches found: {number_of_matches}\n")

df = pd.DataFrame([
    {
        "id": m.id,
        "question": m.question,
        "outcomes": m.outcomes,
        "token_ids": ", ".join(m.metadata.get("clobTokenIds") or []),
    }
    for m in results
])

df.head(5)


Search results for bitcoin (open markets only)
Matches found: 231



,id,question,outcomes,token_ids
0,1214994,"Bitcoin Up or Down - January 19, 3:25PM-3:30PM ET","[Up, Down]",6219449130951924957660560910647254765123352476...
1,1214988,"Bitcoin Up or Down - January 19, 3:20PM-3:25PM ET","[Up, Down]",8641086897223983567196626271828352612965263153...
2,1214979,"Bitcoin Up or Down - January 19, 3:15PM-3:30PM ET","[Up, Down]",7725682069401998694261441589859114100456721450...
3,1214975,"Bitcoin Up or Down - January 19, 3:15PM-3:20PM ET","[Up, Down]",1102722099326970838318873061732960672600643627...
4,1214828,"Bitcoin Up or Down - January 19, 3:10PM-3:15PM ET","[Up, Down]",5521633393371649306750165326976129149927813756...


- use `fetch_public_trades()`to get public trading activity
- first step to hunt for whales/ track which markets are drawing much activity 

In [16]:
# Fetch last 50 public trades
trades = pm_exchange.fetch_public_trades(limit=50)
print("Number of trades fetched:", len(trades))

# Show last 10 public trades as a DataFrame
tdf = (
    pd.DataFrame([
        {
            "timestamp": t.timestamp,
            "side": t.side,
            "price": round(float(t.price), 2),
            "size": round(float(t.size), 2),
            "outcome": t.outcome,
            "title": t.title,
            "slug": t.slug,
            "condition_id": t.condition_id,
        }
        for t in trades
    ])
    .sort_values("timestamp")
)

tdf.tail(10)

Number of trades fetched: 50


,timestamp,side,price,size,outcome,title,slug,condition_id
14,2026-01-18 20:39:04+00:00,SELL,0.01,53.00,Yes,Will Elon Musk post 65-89 tweets from January ...,elon-musk-of-tweets-january-17-january-19-65-89,0x34687e3d176dc89ddad9a6653367d1972ba275e60a09...
15,2026-01-18 20:39:04+00:00,BUY,0.54,13.00,Down,"Ethereum Up or Down - January 18, 3PM ET",ethereum-up-or-down-january-18-3pm-et,0x3104d2b8d01718d2018ffcfc786b238cd3e0a735aa66...
16,2026-01-18 20:39:04+00:00,SELL,0.98,10.00,Down,"Ethereum Up or Down - January 18, 3:30PM-3:45P...",eth-updown-15m-1768768200,0x0aa22ff24b1e3d008ed237d9344ebf2fb6f20fd769fa...
17,2026-01-18 20:39:04+00:00,SELL,0.43,5.00,Yes,"Will the price of Bitcoin be above $96,000 on ...",bitcoin-above-96k-on-january-21,0xc1b46a1d38e0c1d4878e5bc4475079f78ed9b092624d...
18,2026-01-18 20:39:04+00:00,BUY,0.95,15.63,Down,"Bitcoin Up or Down - January 18, 3:30PM-3:45PM ET",btc-updown-15m-1768768200,0xfb84e221ef803f15830591236ea7a4761016a307dcd3...
19,2026-01-18 20:39:04+00:00,SELL,0.59,8.47,Yes,Puffpaw FDV above $50M one day after launch?,puffpaw-fdv-above-50m-one-day-after-launch,0x8bcbc39541f2d12c9e4ec1ac4d181d6aada34852301c...
20,2026-01-18 20:39:04+00:00,BUY,0.62,20.56,Down,"Bitcoin Up or Down - January 18, 3PM ET",bitcoin-up-or-down-january-18-3pm-et,0xa41df28caf1575921c7ab1f4876f75c04a565ee235ec...
21,2026-01-18 20:39:04+00:00,SELL,0.26,4901.25,Yes,"Russia x Ukraine ceasefire by June 30, 2026?",russia-x-ukraine-ceasefire-by-june-30-2026,0xe546672750517f62c45a5a00067481981e62b9c20fa8...
23,2026-01-18 20:39:04+00:00,SELL,0.98,5.00,Down,"Ethereum Up or Down - January 18, 3:30PM-3:45P...",eth-updown-15m-1768768200,0x0aa22ff24b1e3d008ed237d9344ebf2fb6f20fd769fa...
24,2026-01-18 20:39:04+00:00,BUY,0.54,25.00,Down,"Ethereum Up or Down - January 18, 3PM ET",ethereum-up-or-down-january-18-3pm-et,0x3104d2b8d01718d2018ffcfc786b238cd3e0a735aa66...


## Get Price History

- use `fetch_price_history()` to load historical data by market
- more examples on this under backtest/example_ipynb

In [17]:
df_history = pm_exchange.fetch_price_history(
    market,             # <-- Market object (or market.id)
    outcome="Yes", 
    interval="1h",     # supported: 1m, 1h, 6h, 1d, 1w, max
    fidelity=5,        # data point every 5 minutes, lower = more data points
    as_dataframe=True,
)
df_history.tail(20)

,timestamp,price
0,2026-01-18 19:45:14+00:00,0.785
1,2026-01-18 19:50:16+00:00,0.785
2,2026-01-18 19:55:14+00:00,0.785
3,2026-01-18 20:00:22+00:00,0.785
4,2026-01-18 20:05:29+00:00,0.785
5,2026-01-18 20:10:15+00:00,0.785
6,2026-01-18 20:15:13+00:00,0.785
7,2026-01-18 20:20:29+00:00,0.785
8,2026-01-18 20:25:33+00:00,0.785
9,2026-01-18 20:30:34+00:00,0.785
